<a href="https://colab.research.google.com/github/faisalahm3d/Research/blob/master/Bangla%20Fake%20News%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import string
%pip install bangla-stemmer
from bangla_stemmer.stemmer.stemmer import BanglaStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
real = pd.read_csv('/content/drive/MyDrive/Datasets/LabeledAuthentic-7K.csv') # Authentic News
fake = pd.read_csv('/content/drive/MyDrive/Datasets/LabeledFake-1K.csv') # Fake News


In [ ]:
real.drop(['articleID', 'date'], axis = 1, inplace = True)
real['label'] = real['label'].apply(lambda x: int(x))

In [ ]:
fake.drop(['articleID', 'date', 'F-type'], axis = 1, inplace = True)
fake['label'] = fake['label'].apply(lambda x: int(x))

In [ ]:
real.head()

,domain,category,source,relation,headline,content,label
0,bd-pratidin.com,Sports,আফগান ক্রিকেট বোর্ড (এসিবি) প্রধান,Related,হঠাৎ আফগান ক্রিকেট বোর্ড প্রধানের পদত্যাগ,ক্রিকেট বিশ্বের নতুন চমকের নাম আফগানিস্তান। কয়...,1
1,jugantor.com,Sports,Reporter,Related,টস হেরে বোলিংয়ে বাংলাদেশ,এশিয়া কাপের ষষ্ঠ ম্যাচে বাংলাদেশ দলের বিপক্ষে ...,1
2,bd24live.com,National,Reporter,Related,রাজধানীতে বিশেষ অভিযানে আটক ৪৩,রাজধানীতে মাদক বিরোধী বিশেষ অভিযান পরিচালনা কর...,1
3,bd24live.com,National,জননিরাপত্তা বিভাগের সচিব,Related,উস্কানি রোধে নজরদারি থাকবে সামাজিক যোগাযোগ মাধ...,সনাতন ধর্মাবলম্বীদের সবচেয়ে বড় ধর্মীয় উৎসব দুর...,1
4,somoynews.tv,Finance,পেট্রোল পাম্প মালিক সমিতি সভাপতি,Related,"'যেখানে তেল আসত ৭ দিনে, এখন তা আসবে অতি দ্রুত'",ভারত থেকে পাইপ লাইনের মাধ্যমে সরাসরি দেশে জ্বা...,1


In [ ]:
fake.head()

,domain,category,source,relation,headline,content,label
0,channeldhaka.news,International,Reporter,Unrelated,মুরগির হামলায় শেয়াল নিহত,"বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্...",0
1,earki.com,Miscellaneous,Reporter,Unrelated,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,"BTV থেকে লোকজন আসছে, ইন্টারভিউ নিবে।চারজনের টি...",0
2,earki.com,Miscellaneous,Reporter,Unrelated,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,অদ্ভুত বিরোধীদলহীনতায় ভুগছে সরকার। এ এক অন্যরক...,0
3,channeldhaka.news,Sports,Reporter,Unrelated,অবসর নেয়ার ঘোষণা দিলেন মেসি !,রাশিয়া বিশ্বকাপ নকআউট পর্বে ফ্রান্সের সাথে ৪-৩...,0
4,motikontho.wordpress.com,Miscellaneous,Reporter,Unrelated,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...","নিজস্ব মতিবেদক‘মাদারফাকার নহে, আমি ব্রাদারফাকা...",0


In [ ]:
df_merge = pd.concat([fake, real], axis=0)
# df_merge.head(10)

In [ ]:
df_merge.isnull().sum()

domain      0
category    0
source      0
relation    0
headline    0
content     0
label       0
dtype: int64

In [ ]:
df_merge.shape

(8501, 7)

In [ ]:
df = df_merge.sample(frac=1)
# df.head(15)

In [ ]:
### Setting up punctuations, stopwords and numbers ###
punctuations = [',', ';', '।', '?' , '!', ':', '—', ':-', '-', '’', '“', '”', '(', 
')', '{', '}', '[', ']', '√', '<', '>', '=', '...', '.', '/', "'", '"', "‘", '•',
'–', 'é', ' ঃ', '😱', '🙂', '…', '►', '\●', '॥', '¯','œ', '¯', '¯্র',
'Ä', 'ñ', 'ʘ', 'Ñ', 'б', 'а', 'н', 'г', 'л', 'а', 'д', 'е', 'ш', 
'в', 'а', 'у', '😛', '😞', '😥'
]

stopwords = [
    'অবশ্য', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অথবা', 'অথচ',
    'অর্থাত', 'অন্য', 'আজ', 'আছে', 'আপনার', 'আপনি', 'আবার',
    'আগে', 'আগেই', 'আই', 'অতএব', 'আগামী', 'অবধি', 'অনুযায়ী',
    'আদ্যভাগে', 'এই', 'একই', 'একে', 'একটি', 'এখন', 'এখনও', 
    'এখানে', 'এখানেই', 'এটি', 'এটা', 'এটাই', 'এতটাই', 'এবং', 
    'একবার', 'এবার', 'এদের', 'এঁদের', 'এমন', 'এমনকী', 'এল', 
    'এর', 'এরা', 'এঁরা', 'এস', 'এত', 'এতে', 'এসে', 
    'একে', 'এ', 'ঐ', 'ই', 'ইহা', 'ইত্যাদি', 'উনি', 
    'উপর', 'উপরে', 'উচিত', 'ও', 'ওই', 'ওর', 'ওরা', 
    'ওঁর', 'ওঁরা', 'ওকে', 'ওদের', 'ওঁদের', 'ওখানে', 'কত', 
    'কবে', 'করতে', 'কয়েক', 'কয়েকটি', 'করবে', 'করলেন', 'করার', 
    'কারও', 'করা', 'করি', 'করিয়ে', 'করার', 'করাই', 'করলে', 
    'করলেন', 'করিতে', 'করিয়া', 'করেছিলেন', 'করছে', 'করছেন', 'করেছেন', 
    'করেছে', 'করেন', 'করবেন', 'করায়', 'করে', 'করেই', 'কাছ', 
    'কাছে', 'কাজে', 'কারণ', 'কিছু', 'কিছুই', 'কিন্তু', 'কিংবা', 
    'কি', 'কী', 'কেউ', 'কেউই', 'কাউকে', 'কেন', 'কে', 
    'কোনও', 'কোনো', 'কোন', 'কখনও', 'ক্ষেত্রে', 'খুব', 'গুলি', 
    'গিয়ে', 'গিয়েছে', 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 
    'ছাড়া', 'ছাড়াও', 'ছিলেন', 'ছিল', 'জন্য', 'জানা', 'ঠিক', 
    'তিনি', 'তিনঐ', 'তিনিও', 'তখন', 'তবে', 'তবু', 'তাঁদের', 
    'তাঁহারা', 'তাঁরা', 'তাঁর', 'তাঁকে', 'তাই', 'তেমন', 'তাকে', 
    'তাহা', 'তাহাতে', 'তাহার', 'তাদের', 'তারপর', 'তারা', 'তারৈ', 
    'তার', 'তাহলে', 'তিনি', 'তা', 'তাও', 'তাতে', 'তো', 
    'তত', 'তুমি', 'তোমার', 'তথা', 'থাকে', 'থাকা', 'থাকায়', 
    'থেকে', 'থেকেও', 'থাকবে', 'থাকেন', 'থাকবেন', 'থেকেই', 'দিকে', 
    'দিতে', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন', 'দিলেন', 'দু', 'দুটি', 
    'দুটো', 'দেয়', 'দেওয়া', 'দেওয়ার', 'দেখা', 'দেখে', 'দেখতে', 
    'দ্বারা', 'ধরে', 'ধরা', 'নয়', 'নানা', 'না', 'নাকি', 
    'নাগাদ', 'নিতে', 'নিজে', 'নিজেই', 'নিজের', 'নিজেদের', 'নিয়ে', 
    'নেওয়া', 'নেওয়ার', 'নেই', 'নাই', 'পক্ষে', 'পর্যন্ত', 'পাওয়া', 
    'পারেন', 'পারি', 'পারে', 'পরে', 'পরেই', 'পরেও', 'পর', 
    'পেয়ে', 'প্রতি', 'প্রভৃতি', 'প্রায়', 'ফের', 'ফলে', 'ফিরে', 
    'ব্যবহার', 'বলতে', 'বললেন', 'বলেছেন', 'বলল', 'বলা', 'বলেন', 
    'বলে', 'বহু', 'বসে', 'বার', 'বা', 'বিনা', 'বরং', 
    'বদলে', 'বাদে', 'বার', 'বিশেষ', 'বিভিন্ন', 'বিষয়টি', 'ব্যবহার', 
    'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মধ্যভাগে', 
    'মাধ্যমে', 'মাত্র', 'মতো', 'মতোই', 'মোটেই', 'যখন', 'যদি', 
    'যদিও', 'যাবে', 'যায়', 'যাকে', 'যাওয়া', 'যাওয়ার', 'যত', 
    'যতটা', 'যা', 'যার', 'যারা', 'যাঁর', 'যাঁরা', 'যাদের', 
    'যান', 'যাচ্ছে', 'যেতে', 'যাতে', 'যেন', 'যেমন', 'যেখানে', 
    'যিনি', 'যে', 'রেখে', 'রাখা', 'রয়েছে', 'রকম', 'শুধু', 
    'সঙ্গে', 'সঙ্গেও', 'সমস্ত', 'সব', 'সবার', 'সহ', 'সুতরাং', 
    'সহিত', 'সেই', 'সেটা', 'সেটি', 'সেটাই', 'সেটাও', 'সম্প্রতি', 
    'সেখান', 'সেখানে', 'সে', 'স্পষ্ট', 'স্বয়ং', 'হইতে', 'হইবে', 
    'হৈলে', 'হইয়া', 'হচ্ছে', 'হত', 'হতে', 'হতেই', 'হবে', 
    'হবেন', 'হয়েছিল', 'হয়েছে', 'হয়েছেন', 'হয়ে', 'হয়নি', 'হয়', 
    'হয়েই', 'হয়তো', 'হল', 'হলে', 'হলেই', 'হলেও', 'হলো', 
    'হিসাবে', 'হওয়া', 'হওয়ার', 'হওয়ায়', 'হন', 'হোক', 'জন', 
    'জনকে', 'জনের', 'জানতে', 'জানায়', 'জানিয়ে', 'জানানো', 'জানিয়েছে', 
    'জন্য', 'জন্যওজে', 'জে', 'বেশ', 'দেন', 'তুলে', 'ছিলেন', 
    'চান', 'চায়', 'চেয়ে', 'মোট', 'যথেষ্ট', 'টি'
]

stemmer = BanglaStemmer()
# numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
# alphabets = string.ascii_lowercase + string.ascii_uppercase
# alphabets_list = list(alphabets)

In [ ]:
def remove_punctuation(txt):
    txt_nopunc = "".join([c for c in txt if c not in punctuations])
    return txt_nopunc

def remove_english_punctuation(txt):
    txt_nopunc = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunc

def tokenize(txt):
    tokens = re.split('\s+', txt)
    return tokens

def remove_stopwords(txt_tokenized):
    txt_clean = ([word for word in txt_tokenized if word not in stopwords])
    return txt_clean

def stemming(txt_no_sw):
    stemmed_txt = [stemmer.stem(word) for word in txt_no_sw]
    return stemmed_txt

In [ ]:
df['category'] = df['category'].apply(lambda x: x.lower())
df['source'] = df['source'].apply(lambda x: x.lower())
df['relation'] = df['relation'].apply(lambda x: x.lower())

In [ ]:
df['news'] = df['headline'] + ' ' + df['content']

In [ ]:
df.drop(['headline', 'content'], axis = 1, inplace = True)

In [ ]:
df['news'] = df.news.apply(lambda x: remove_english_punctuation(x))
df['news'] = df.news.apply(lambda x: remove_punctuation(x))
df['news'] = df.news.apply(lambda x: ' '.join(x.split()))
df['news'] = df.news.apply(lambda x: tokenize(x))
df['news'] = df.news.apply(lambda x: remove_stopwords(x))
df['news'] = df.news.apply(lambda x: stemming(x))
df['news'] = df.news.apply(lambda x: ' '.join(x))

In [ ]:
df.head(10)

,domain,category,source,relation,label,news
1598,dhakatimes24.com,sports,reporter,related,1,ভারতপাকিস্তান মহারণ বিশ্ব ক্রিকেট সবসময় চিরপ্র...
702,motikontho.wordpress.com,national,reporter,unrelated,0,চট্টগ্রাম মেডিকেলের শিক্ষার্থী সড়ক অবরোধ দৈনিক...
6628,jagonews24.com,national,বগুড়া রেলওয়ের স্টেশন মাস্টার,related,1,দাব সেতু বগুড়াসান্ত রুটে ট্র চলাচল বন্ধ সেতু দ...
4887,banglatribune.com,international,ওয়ানজা বন্দরর আঞ্চলিক পুলিশ কমান্ডার,related,1,তানজানিয়ায় ফেরি উল্টে নিহত শতাধিক নিখোঁজ দুইশ ...
5744,independent24.com,politics,আওয়ামী লীগ সাধারণ সম্পাদক,related,1,দেশত্যাগে বাধ্য অভিযোগ মনগড়া কাদের সাবেক প্রধা...
6891,independent24.com,national,reporter,related,1,মধ্যরাত শুরু ইলিশ ধরায় নিষেধাজ্ঞা প্রজনন বাড়া ...
538,rtnn.net,international,বিবিসি,related,1,তাঞ্জানিয়ায় ফেরি ডুবে নিহত ৪৪ আন্তর্জাতিক ডেস্...
4504,bd-pratidin.com,sports,reporter,related,1,মেহেদিমাশরাফির লড়া ১৭৩ রান সংগ্রহ টাইগার চলতি ...
1071,motikontho.wordpress.com,miscellaneous,reporter,unrelated,0,জন্মদিনে বাবুনগরী লও লও লও ছাল ওবামা দৈনিক মতি...
3631,banglatribune.com,national,চট্টগ্রাম বিশ্ববিদ্যালয় ছাত্রলীগের সাবেক সহ-সভ...,related,1,সাকা চৌধুরীর কবর শহীদ লেখা নামফলক অপসারণ করলো ...


In [ ]:
x = df.drop(['label'], axis = 1) # dropping the target
y = df['label']

In [ ]:
x['test'] = x.domain + ' ' +  x.category + ' ' + x.source + ' ' + x.relation + ' ' + x.news

In [ ]:
x = x.drop(['domain', 'category', 'source', 'relation', 'news'], axis=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
vect = TfidfVectorizer(encoding="utf-8", analyzer=tokenize)
xv = vect.fit_transform(x.test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(xv, y, test_size=0.3, random_state=0, stratify=y)

Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(X_train, y_train)

MultinomialNB()

In [ ]:
NB.score(X_test, y_test)

0.8522148177185418

Classification Report of Naive Bayes

In [ ]:
pred_NB = NB.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_NB)) 

              precision    recall  f1-score   support

           0       0.88      0.04      0.07       390
           1       0.85      1.00      0.92      2161

    accuracy                           0.85      2551
   macro avg       0.87      0.52      0.50      2551
weighted avg       0.86      0.85      0.79      2551



Confusion Matrix NB

In [ ]:
print(confusion_matrix(y_test, pred_NB))

[[  15  375]
 [   2 2159]]


Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [ ]:
LR.score(X_test, y_test)

0.9368874950999608

Classification Report of Logistic Regression

In [ ]:
pred_LR = LR.predict(X_test)

In [ ]:
print(classification_report(y_test, pred_LR)) 

              precision    recall  f1-score   support

           0       0.97      0.61      0.75       390
           1       0.93      1.00      0.96      2161

    accuracy                           0.94      2551
   macro avg       0.95      0.80      0.85      2551
weighted avg       0.94      0.94      0.93      2551



Confusion Matrix LR

In [ ]:
print(confusion_matrix(y_test, pred_LR)) # very poor results for minority class

[[ 236  154]
 [   7 2154]]


Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(random_state=0)

In [ ]:
DT.fit(X_train, y_train)
DT.score(X_test, y_test)

0.9921599372794982

Classification Report of Decision Tree

In [ ]:
pred_DT = DT.predict(X_test)
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       390
           1       0.99      1.00      1.00      2161

    accuracy                           0.99      2551
   macro avg       0.99      0.98      0.98      2551
weighted avg       0.99      0.99      0.99      2551



Confusion Matrix DT

In [ ]:
print(confusion_matrix(y_test, pred_DT)) # Way to go DT!!

[[ 376   14]
 [   6 2155]]


KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KN = KNeighborsClassifier(n_neighbors=93)

In [ ]:
KN.fit(X_test, y_test)
KN.score(X_test, y_test)

0.8769110152881223

Classification Report of KNN

In [ ]:
pred_KN = KN.predict(X_test)
print(classification_report(y_test, pred_KN))

              precision    recall  f1-score   support

           0       1.00      0.19      0.33       390
           1       0.87      1.00      0.93      2161

    accuracy                           0.88      2551
   macro avg       0.94      0.60      0.63      2551
weighted avg       0.89      0.88      0.84      2551



Confusion Matrix KNN

In [ ]:
print(confusion_matrix(y_test, pred_KN)) # Poor results for minority class

[[  76  314]
 [   0 2161]]


In [ ]:
news = [
    'bengalbeats.com miscellaneous reporter unrelated জেলে খাই বেহেস্তি খানা ট্রাইবুনালে যাওয়া মানা নিজামী দৈনিক মতিকণ্ঠ কারাগার মতিবেদকঢাকা কেন্দ্রীয় কারাগারে উচ্চ রক্তচাপে আক্রান্ত বৃহত্তর জামায়াতে সলামীর খানকির পোলায়ে আমীর  কাত্তরে আল বদরের কমান্ডার মতিউর রহমান নিজামী বলেছেন জেলে খা বেহেস্তি খানা ট্রাবুনালে তা যায়া মানামংগলবার সন্ধায় কেন্দ্রীয় কারাগারে আয়জিত ক সংবাদ সম্মেলনে আল বদর কমান্ডার নিজামী  কথা বলেন সময় তাকে হাস্যজ্জল দেখাচ্ছিলসংবাদ সম্মেলনে নিজামী বলেন আমাদের কারাগারগুলিতে খানাপিনার বেবস্থা মাশাল্লাহ ফাশ কিলাশ কাশিমপুর কারাগারে সকালের নাশতায় আমায় দেয়া হত লাল আটার পাঁচটি রুটি সঙ্গে ডিম ভাজি আলু ছাড়া সবজি ভাজি মুরগির মাংস মিষ্টি নসির দুধ  কলা নাশতা খায়া হজম করিয়া সারতারি না তার আগে চলিয়া আসিত দুপুরের খাবারের চিকন চালের ভাত করলা ভাজি টাকি বা চিংড়ি মাছ ভর্তা বেগুন ভাজি বা ভর্তা ছোট মাছ বা চিংড়ি মাছ ভুনা সালাদ  লেবু মাল্টা বা বর বা নাশপাতি তারপর পেট ভরিয়া খানাদানার পর দিতাম ঘুম ঘুম ভাংগিয়া সন্ধ্যার খাবারে খাতাম লাড্ডু নিমকিবিস্কুট  হরলিকস বা স্যুপ তারপর কারাগারে কুন কচি মোজাহিদের সংগে নিবিড় বেয়াম করতাম কিছুক্ষন রাতের খাবারে আসিত চিকন চালের ভাত করলা ভাজি বেগুন ভাজি বা ভর্তা ঢেড়শ বা মিষ্টিকুমড়া বা পেঁপে ভাজি গরু বা খাসির মাংস ভুনা সালাদ  লেবু বং কমলা মাল্টা নাশপাতি আঙুর বা বরআবেগঘন কণ্ঠে নিজামী বলেন আমি জানি বেহেস্তে যায়ার কুন সুযুগ আমার না আমার বিপদ ফাঁসির পর কবরে আযাব দিয়া শুরু হবে দোজখের আগুনে জ্বলিয়া পুড়িয়া আখেরাতে গিয়া শেষ হবে  সুদির্ঘ জীবনে কারাগারে কাটান বতসরগুলু আমার জীবনের শ্রেষ্ঠ সময়  যেন দুনিয়ার বুকে ক টুকরু শাদ্দাদের বেহেস্ত খায়া শোয়ার অপুর্ব বন্দবস্তহাসতে হাসতে নিজামী বলেন রুপ খানাপিনা করলে রক্তচাপ বৃদ্দি পায়া স্বাভাবিক তা আজ সকালে ডাক্তারকে ডাকিয়া বললাম ডাক্তার ডাক্তার শুন না ত বেশী ভাত খায়া ভাল না ডাক্তার আসিয়া আমায় কিছুক্ষন টিপিয়া টুপিয়া চলিয়া গেল কারাগারের নায়েবে আমীর ফরমান আলী ছার আসিয়া আমায় বললেন নিজামী সাব আজকের দিনটা রেষ্টে থাকেন ট্রাবুনালে গিয়া কাম নাঅশ্রু মুছে নিজামী বলেন কারাগারের নায়েব ফরমান আলী ছারের মত ভাল বেক্তি দানীং আর দেখা যায় না কাত্তর সালে আরেক ফরমান আলীর আন্ডারে কিছুদিন কাম করার অভিজ্ঞতা হছিল সে ফরমান আলীর পর চারটি দশক কাটিয়া গেল তারপর পালাম  ফরমান আলীরে তার নাম শুনিয়া আমার রক্তচাপ দেড় পেসকেল কমিয়া গিয়াছেঅচিরে সুস্থ হয়ে ট্রাবুনালের রায় শ্রবনের চ্ছা বেক্ত করে নিজামী বলেন শুনিলাম সলামী ছাত্র সংস্থার ২৪টি যুবতী ফিপটি শেডস অফ গ্রে সহ ধরা পড়িয়াছে উহাদের আমার কারাকক্ষে পাঠান হলে শাদ্দাদের বেহেস্ত দেখা ষুল আনা পুরন হত',
    'motichurladdoo.com national reporter unrelated এবার মশার বিরুদ্ধে গর্জে উঠলো ছাত্রলীগের কামান সম্প্রতি ঢাকা বিশ্ববিদ্যালয় প্রাঙ্গণে ছাত্রলীগের কটি অংশকে মশা মারার যন্ত্র নিয়ে ঘুরে ঘুরে মশা মারতে দেখা যায় বিস্তারিত খোঁজ করতে গেলে জানা যায় ছাত্রলীগের সাধারণ সম্পাদক গোলাম রাব্বানী ডেঙ্গু জ্বরে আক্রান্ত হবার পর ঢাবি ক্যাম্পাসে মশা নিধন কর্মসূচি নিয়েছে সংগঠনটির নেতাকর্মীরাগতকাল ২৪ নভেম্বর সন্ধ্যায় ছাত্রলীগের সভাপতি সঞ্জিত চন্দ্র দাসের নেতৃত্বে মধুর ক্যান্টিন থেকে  মশা নিধন অভিযান শুরু হয় ক্যাম্পাসের বিভিন্ন গুরুত্বপূর্ণ স্থান বং হলগুলোর আশেপাশে মশা মারার ষুধ দেন তারা ব্যাপারে সংগঠনটির কজন কর্মীর কাছে জানতে চায়া হলে তিনি মশা মারতে ব্যস্ত আছেন বলে জানান পাশে থাকা তার কজন সহমত ভা আমাদের বলেন বাংলাদেশে মশার কোন স্থান নে মশা মন কটা পতঙ্গ যাকে বাংলার মানুষ চায় না আমরা অনেক আগে থেকে  কর্মসূচি নেবার কথা চিন্তা করছিলাম কিন্তু বিভিন্ন নির্বাচনী ব্যস্ততায় করা হয়ে ঠেনি তবে বার গোলাম রাব্বানী ভায়ের অসুস্থতার পর আমরা আর বসে থাকতে পারলাম নাসহমত ভাটি আর জানান কথায় আছে মশা মারতে কামান লাগে সিটি কর্পোরেশন যে তোদিন ধরে মশা দমন কর্মসূচি পালন করে আসলো কিন্তু তাতে কোন লাভ না হয়াতে বাধ্য হয়ে আমাদের নিজস্ব স্পেশাল কামান দিয়ে  দমন কর্মসূচি শুরু করা হয়েছেআরেকজন সহমত ভা জানান আরো কটি চাঞ্চল্যকর তথ্য  দমন অভিযানকে সফল করার জন্যে ছাত্রলীগ কটি আলাদা সংগঠন খোলার কথা চিন্তা করছে সংগঠনটির নাম হবে মশকী দমন দপ্তর রফে মদদ তিনি উচ্ছ্বসিত হয়ে আরো বলেন বিরোধী দল বারবার আমাদেরকে মদদপুষ্ট দল হিসেবে আখ্যা দিয়ে সেছে না গেস হোয়াট আমরা খন আসলে মদদয়ালা কটা দল হিসেবে আত্মপ্রকাশ করতে চলেছি পর্যায়ে নিজের গালে বসা কটি মশাকে মারতে গিয়ে তিনি কিঞ্চিত আঘাতপ্রাপ্ত হন কটু সামলে নিয়ে তিনি যোগ করেন আমরা সবাকে কটা মেসেজ দিতে চা যে সকল মদদ খারাপ না কিছু কিছু মদদ দেশ  দশের কাজে লাগে ছাড়া আমরা কঠোর হাতে  দমন কর্মসূচি চালিয়ে যাবোতার বক্তব্য শেষ হয়ার সঙ্গে সঙ্গে অন্য কজন জুনিয়র সহমত ভা সহমত পোষণ করে স্লোগানের সুরে বলেন মশকীর আস্তানা বাংলাদেশে থাকবে না দেশের সাধারণ জনগণের প্রতি আমাদের আকুতি দয়া করে আপনারা মশা মারুন আপনারা মশা না মারলে মশা আপনাকে মেরে ফেলবে দয়া করে ঘুমিয়ে থাকবেন না উঠুন জাগুন প্রয়োজনে মশা মারতে কামান দাগুনতবে নাম প্রকাশে অনিচ্ছুক ক অসহমত ভা আমাদের প্রতিবেদককে কিছুটা নিচু স্বরে জানিয়েছেন রাব্বানী ভা কবার বিজয় সরণির জ্যামে পড়লে আশা করি আমরা বিপুল নেতাকর্মীকে বিজয় সরণিতে জ্যাম সমস্যা সমাধানের লক্ষ্যে মাঠে নামতে দেখতে পাবো',
    'jagonews24.com national reporter related রেমিটেন্স অ্যাওয়ার্ড পেলো ইসলামী ব্যাংক সর্বোচ্চ রেমিটেন্স আহরণের জন্য বাংলাদেশ ব্যাংক রেমিটেন্স অ্যায়ার্ড ২০১৭ লাভ করেছে সলামী ব্যাংক বাংলাদেশ লিমিটেড বৃহস্পতিবার ২০ সেপ্টেম্বর বাংলাদেশ ব্যাংক ট্রেনিং কাডেমি মিলনায়তনে ক অনুষ্ঠানে ব্যাংকের ম্যানেজিং ডিরেক্টর মডি প্রধান নির্বাহী সি মো মাহবুব উল আলমের হাতে পুরস্কার তুলে দেন অর্থমন্ত্রী আবুল মাল আব্দুল মুহিত বাংলাদেশ ব্যাংকের নির্বাহী পরিচালক মো আব্দুর রহিম সভাপতিত্বে অনুষ্ঠানে বিশেষ অতিথি হিসেবে উপস্থিত ছিলেনবাংলাদেশ ব্যাংকের গভর্ণর ফজলে কবির অর্থমন্ত্রণালয়ের আর্থিক প্রতিষ্ঠান বিভাগের অতিরিক্ত সচিব কামরুন নাহার আহমেদ প্রবাসী কল্যাণ বৈদেশিক কর্মসংস্থান মন্ত্রণালয়ের অতিরিক্ত সচিব ড আহমেদ মুনিরুছ সালেহীন সময় সলামী ব্যাংকের ডেপুটি ম্যানেজিং ডিরেক্টর ন্টারন্যাশনাল ব্যাংকিং উং প্রধান হাসনে আলমসহ বিভিন্ন ব্যাংকের ঊর্ধ্বতন নির্বাহী প্রবাসীরা উপস্থিত ছিলেন'
]
news_count = vect.transform(news)

Prediction [0 for Fake, 1 for Real]

In [ ]:
NB.predict(news_count)

array([1, 1, 1])

In [ ]:
LR.predict(news_count)

array([0, 1, 1])

In [ ]:
DT.predict(news_count) # Decision Tree got it right

array([0, 0, 1])

In [ ]:
KN.predict(news_count) 

array([0, 1, 1])

In [ ]:
y_train.value_counts() # Imbalance :/

1    5041
0     909
Name: label, dtype: int64

Creating Synthetic Fake News To Handle Imbalance

In [ ]:
# !pip install imblearn
from imblearn.over_sampling import SMOTE # takes avg of neighboring data and creates new data
smote = SMOTE(sampling_strategy='minority')

In [ ]:
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
print('Before SMOTE: ', Counter(y_train))
print('After SMOTE: ', Counter(y_train_smote)) # Balanced Yahoo!!!

Before SMOTE:  Counter({1: 5041, 0: 909})
After SMOTE:  Counter({1: 5041, 0: 5041})


Testing out if performance increased

In [ ]:
NB.fit(X_train_smote, y_train_smote)
NB.score(X_test, y_test)

0.9290474323794591

In [ ]:
pred_NB = NB.predict(X_test)
print(classification_report(y_test, pred_NB))

              precision    recall  f1-score   support

           0       0.71      0.91      0.80       390
           1       0.98      0.93      0.96      2161

    accuracy                           0.93      2551
   macro avg       0.85      0.92      0.88      2551
weighted avg       0.94      0.93      0.93      2551



In [ ]:
print(confusion_matrix(y_test, pred_NB))

[[ 354   36]
 [ 145 2016]]


In [ ]:
LR.fit(X_train_smote, y_train_smote)
LR.score(X_test, y_test)

0.9627597020776166

In [ ]:
pred_LR = LR.predict(X_test)
print(classification_report(y_test, pred_LR)) 

              precision    recall  f1-score   support

           0       0.85      0.92      0.88       390
           1       0.98      0.97      0.98      2161

    accuracy                           0.96      2551
   macro avg       0.92      0.94      0.93      2551
weighted avg       0.96      0.96      0.96      2551



In [ ]:
print(confusion_matrix(y_test, pred_LR)) # pretty decent performance boost for Naive Bayes and Logistic Regression

[[ 358   32]
 [  63 2098]]


In [ ]:
DT.fit(X_train_smote, y_train_smote)
DT.score(X_test, y_test)

0.9925519404155233

In [ ]:
pred_DT = DT.predict(X_test)
print(classification_report(y_test, pred_DT)) 

              precision    recall  f1-score   support

           0       0.99      0.96      0.98       390
           1       0.99      1.00      1.00      2161

    accuracy                           0.99      2551
   macro avg       0.99      0.98      0.99      2551
weighted avg       0.99      0.99      0.99      2551



In [ ]:
print(confusion_matrix(y_test, pred_DT)) # Performance kind of remained same, still the best 😎

[[ 375   15]
 [   4 2157]]


In [ ]:
KN.fit(X_train_smote, y_train_smote)
KN.score(X_test, y_test)

0.23167385339082713

In [ ]:
pred_KN = KN.predict(X_test)
print(classification_report(y_test, pred_KN)) 

              precision    recall  f1-score   support

           0       0.17      1.00      0.28       390
           1       1.00      0.09      0.17      2161

    accuracy                           0.23      2551
   macro avg       0.58      0.55      0.23      2551
weighted avg       0.87      0.23      0.19      2551



In [ ]:
print(confusion_matrix(y_test, pred_KN)) # Performance decreased after smote for this specific model!

[[ 389    1]
 [1959  202]]


In [ ]:
news = [
    'bengalbeats.com miscellaneous reporter unrelated জেলে খাই বেহেস্তি খানা ট্রাইবুনালে যাওয়া মানা নিজামী দৈনিক মতিকণ্ঠ কারাগার মতিবেদকঢাকা কেন্দ্রীয় কারাগারে উচ্চ রক্তচাপে আক্রান্ত বৃহত্তর জামায়াতে সলামীর খানকির পোলায়ে আমীর  কাত্তরে আল বদরের কমান্ডার মতিউর রহমান নিজামী বলেছেন জেলে খা বেহেস্তি খানা ট্রাবুনালে তা যায়া মানামংগলবার সন্ধায় কেন্দ্রীয় কারাগারে আয়জিত ক সংবাদ সম্মেলনে আল বদর কমান্ডার নিজামী  কথা বলেন সময় তাকে হাস্যজ্জল দেখাচ্ছিলসংবাদ সম্মেলনে নিজামী বলেন আমাদের কারাগারগুলিতে খানাপিনার বেবস্থা মাশাল্লাহ ফাশ কিলাশ কাশিমপুর কারাগারে সকালের নাশতায় আমায় দেয়া হত লাল আটার পাঁচটি রুটি সঙ্গে ডিম ভাজি আলু ছাড়া সবজি ভাজি মুরগির মাংস মিষ্টি নসির দুধ  কলা নাশতা খায়া হজম করিয়া সারতারি না তার আগে চলিয়া আসিত দুপুরের খাবারের চিকন চালের ভাত করলা ভাজি টাকি বা চিংড়ি মাছ ভর্তা বেগুন ভাজি বা ভর্তা ছোট মাছ বা চিংড়ি মাছ ভুনা সালাদ  লেবু মাল্টা বা বর বা নাশপাতি তারপর পেট ভরিয়া খানাদানার পর দিতাম ঘুম ঘুম ভাংগিয়া সন্ধ্যার খাবারে খাতাম লাড্ডু নিমকিবিস্কুট  হরলিকস বা স্যুপ তারপর কারাগারে কুন কচি মোজাহিদের সংগে নিবিড় বেয়াম করতাম কিছুক্ষন রাতের খাবারে আসিত চিকন চালের ভাত করলা ভাজি বেগুন ভাজি বা ভর্তা ঢেড়শ বা মিষ্টিকুমড়া বা পেঁপে ভাজি গরু বা খাসির মাংস ভুনা সালাদ  লেবু বং কমলা মাল্টা নাশপাতি আঙুর বা বরআবেগঘন কণ্ঠে নিজামী বলেন আমি জানি বেহেস্তে যায়ার কুন সুযুগ আমার না আমার বিপদ ফাঁসির পর কবরে আযাব দিয়া শুরু হবে দোজখের আগুনে জ্বলিয়া পুড়িয়া আখেরাতে গিয়া শেষ হবে  সুদির্ঘ জীবনে কারাগারে কাটান বতসরগুলু আমার জীবনের শ্রেষ্ঠ সময়  যেন দুনিয়ার বুকে ক টুকরু শাদ্দাদের বেহেস্ত খায়া শোয়ার অপুর্ব বন্দবস্তহাসতে হাসতে নিজামী বলেন রুপ খানাপিনা করলে রক্তচাপ বৃদ্দি পায়া স্বাভাবিক তা আজ সকালে ডাক্তারকে ডাকিয়া বললাম ডাক্তার ডাক্তার শুন না ত বেশী ভাত খায়া ভাল না ডাক্তার আসিয়া আমায় কিছুক্ষন টিপিয়া টুপিয়া চলিয়া গেল কারাগারের নায়েবে আমীর ফরমান আলী ছার আসিয়া আমায় বললেন নিজামী সাব আজকের দিনটা রেষ্টে থাকেন ট্রাবুনালে গিয়া কাম নাঅশ্রু মুছে নিজামী বলেন কারাগারের নায়েব ফরমান আলী ছারের মত ভাল বেক্তি দানীং আর দেখা যায় না কাত্তর সালে আরেক ফরমান আলীর আন্ডারে কিছুদিন কাম করার অভিজ্ঞতা হছিল সে ফরমান আলীর পর চারটি দশক কাটিয়া গেল তারপর পালাম  ফরমান আলীরে তার নাম শুনিয়া আমার রক্তচাপ দেড় পেসকেল কমিয়া গিয়াছেঅচিরে সুস্থ হয়ে ট্রাবুনালের রায় শ্রবনের চ্ছা বেক্ত করে নিজামী বলেন শুনিলাম সলামী ছাত্র সংস্থার ২৪টি যুবতী ফিপটি শেডস অফ গ্রে সহ ধরা পড়িয়াছে উহাদের আমার কারাকক্ষে পাঠান হলে শাদ্দাদের বেহেস্ত দেখা ষুল আনা পুরন হত',
    'motichurladdoo.com national reporter unrelated এবার মশার বিরুদ্ধে গর্জে উঠলো ছাত্রলীগের কামান সম্প্রতি ঢাকা বিশ্ববিদ্যালয় প্রাঙ্গণে ছাত্রলীগের কটি অংশকে মশা মারার যন্ত্র নিয়ে ঘুরে ঘুরে মশা মারতে দেখা যায় বিস্তারিত খোঁজ করতে গেলে জানা যায় ছাত্রলীগের সাধারণ সম্পাদক গোলাম রাব্বানী ডেঙ্গু জ্বরে আক্রান্ত হবার পর ঢাবি ক্যাম্পাসে মশা নিধন কর্মসূচি নিয়েছে সংগঠনটির নেতাকর্মীরাগতকাল ২৪ নভেম্বর সন্ধ্যায় ছাত্রলীগের সভাপতি সঞ্জিত চন্দ্র দাসের নেতৃত্বে মধুর ক্যান্টিন থেকে  মশা নিধন অভিযান শুরু হয় ক্যাম্পাসের বিভিন্ন গুরুত্বপূর্ণ স্থান বং হলগুলোর আশেপাশে মশা মারার ষুধ দেন তারা ব্যাপারে সংগঠনটির কজন কর্মীর কাছে জানতে চায়া হলে তিনি মশা মারতে ব্যস্ত আছেন বলে জানান পাশে থাকা তার কজন সহমত ভা আমাদের বলেন বাংলাদেশে মশার কোন স্থান নে মশা মন কটা পতঙ্গ যাকে বাংলার মানুষ চায় না আমরা অনেক আগে থেকে  কর্মসূচি নেবার কথা চিন্তা করছিলাম কিন্তু বিভিন্ন নির্বাচনী ব্যস্ততায় করা হয়ে ঠেনি তবে বার গোলাম রাব্বানী ভায়ের অসুস্থতার পর আমরা আর বসে থাকতে পারলাম নাসহমত ভাটি আর জানান কথায় আছে মশা মারতে কামান লাগে সিটি কর্পোরেশন যে তোদিন ধরে মশা দমন কর্মসূচি পালন করে আসলো কিন্তু তাতে কোন লাভ না হয়াতে বাধ্য হয়ে আমাদের নিজস্ব স্পেশাল কামান দিয়ে  দমন কর্মসূচি শুরু করা হয়েছেআরেকজন সহমত ভা জানান আরো কটি চাঞ্চল্যকর তথ্য  দমন অভিযানকে সফল করার জন্যে ছাত্রলীগ কটি আলাদা সংগঠন খোলার কথা চিন্তা করছে সংগঠনটির নাম হবে মশকী দমন দপ্তর রফে মদদ তিনি উচ্ছ্বসিত হয়ে আরো বলেন বিরোধী দল বারবার আমাদেরকে মদদপুষ্ট দল হিসেবে আখ্যা দিয়ে সেছে না গেস হোয়াট আমরা খন আসলে মদদয়ালা কটা দল হিসেবে আত্মপ্রকাশ করতে চলেছি পর্যায়ে নিজের গালে বসা কটি মশাকে মারতে গিয়ে তিনি কিঞ্চিত আঘাতপ্রাপ্ত হন কটু সামলে নিয়ে তিনি যোগ করেন আমরা সবাকে কটা মেসেজ দিতে চা যে সকল মদদ খারাপ না কিছু কিছু মদদ দেশ  দশের কাজে লাগে ছাড়া আমরা কঠোর হাতে  দমন কর্মসূচি চালিয়ে যাবোতার বক্তব্য শেষ হয়ার সঙ্গে সঙ্গে অন্য কজন জুনিয়র সহমত ভা সহমত পোষণ করে স্লোগানের সুরে বলেন মশকীর আস্তানা বাংলাদেশে থাকবে না দেশের সাধারণ জনগণের প্রতি আমাদের আকুতি দয়া করে আপনারা মশা মারুন আপনারা মশা না মারলে মশা আপনাকে মেরে ফেলবে দয়া করে ঘুমিয়ে থাকবেন না উঠুন জাগুন প্রয়োজনে মশা মারতে কামান দাগুনতবে নাম প্রকাশে অনিচ্ছুক ক অসহমত ভা আমাদের প্রতিবেদককে কিছুটা নিচু স্বরে জানিয়েছেন রাব্বানী ভা কবার বিজয় সরণির জ্যামে পড়লে আশা করি আমরা বিপুল নেতাকর্মীকে বিজয় সরণিতে জ্যাম সমস্যা সমাধানের লক্ষ্যে মাঠে নামতে দেখতে পাবো',
    'jagonews24.com national reporter related রেমিটেন্স অ্যাওয়ার্ড পেলো ইসলামী ব্যাংক সর্বোচ্চ রেমিটেন্স আহরণের জন্য বাংলাদেশ ব্যাংক রেমিটেন্স অ্যায়ার্ড ২০১৭ লাভ করেছে সলামী ব্যাংক বাংলাদেশ লিমিটেড বৃহস্পতিবার ২০ সেপ্টেম্বর বাংলাদেশ ব্যাংক ট্রেনিং কাডেমি মিলনায়তনে ক অনুষ্ঠানে ব্যাংকের ম্যানেজিং ডিরেক্টর মডি প্রধান নির্বাহী সি মো মাহবুব উল আলমের হাতে পুরস্কার তুলে দেন অর্থমন্ত্রী আবুল মাল আব্দুল মুহিত বাংলাদেশ ব্যাংকের নির্বাহী পরিচালক মো আব্দুর রহিম সভাপতিত্বে অনুষ্ঠানে বিশেষ অতিথি হিসেবে উপস্থিত ছিলেনবাংলাদেশ ব্যাংকের গভর্ণর ফজলে কবির অর্থমন্ত্রণালয়ের আর্থিক প্রতিষ্ঠান বিভাগের অতিরিক্ত সচিব কামরুন নাহার আহমেদ প্রবাসী কল্যাণ বৈদেশিক কর্মসংস্থান মন্ত্রণালয়ের অতিরিক্ত সচিব ড আহমেদ মুনিরুছ সালেহীন সময় সলামী ব্যাংকের ডেপুটি ম্যানেজিং ডিরেক্টর ন্টারন্যাশনাল ব্যাংকিং উং প্রধান হাসনে আলমসহ বিভিন্ন ব্যাংকের ঊর্ধ্বতন নির্বাহী প্রবাসীরা উপস্থিত ছিলেন'
]
news_count = vect.transform(news)

Let's predict again

In [ ]:
NB.predict(news_count) # Predicted Right

array([0, 0, 1])

In [ ]:
LR.predict(news_count) # Predicted Right

array([0, 0, 1])

In [ ]:
DT.predict(news_count) # Predicted Right

array([0, 0, 1])

In [ ]:
KN.predict(news_count) # Definite No GO!!!

array([0, 0, 0])

Final Section

In [ ]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"
    
def manual_testing(news):
    news_test = {'text': [news]}
    test_df = pd.DataFrame(news_test)
    test_df.text = test_df['text'].apply(remove_punctuation)
    test_df.text = test_df['text'].apply(remove_english_punctuation)    
    test_df.text = test_df['text'].apply(lambda x: x.lower())
    test_df.text = test_df['text'].apply(tokenize)
    test_df.text = test_df['text'].apply(remove_stopwords)
    test_df.text = test_df['text'].apply(stemming)
    test_df.text = test_df['text'].apply(remove_stopwords)    
    test_df.text = test_df['text'].apply(lambda x: ' '.join(x))
    test_df.text = test_df['text'].apply(lambda x: ' '.join(x.split()))
    new_xv_test = vect.transform(test_df.text)
    pred_NB = NB.predict(new_xv_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_KN = KN.predict(new_xv_test)
    return print('\n\nNB Prediction: {} \nLR Prediction: {} \nDT Prediction: {} \nKN Prediction: {}'.format(output_label(pred_NB), 
                                                                                                            output_label(pred_LR), 
                                                                                                            output_label(pred_DT), 
                                                                                                            output_label(pred_KN)))

In [ ]:
inp_domain = str(input("Enter domain: "))
inp_category = str(input("Enter category: "))
inp_source = str(input("Enter source: "))
inp_relation = str(input("Enter relation: "))
inp_title = str(input("Enter title: "))
inp_content = str(input("Enter content: "))
inp_news = inp_domain + ' ' + inp_category + ' ' + inp_source + ' ' + inp_relation + ' ' + inp_title + ' ' + inp_content
manual_testing(inp_news)

Enter domain: blackforest.com
Enter category: Black Gold
Enter source: Gold Digger
Enter relation: Unrelated
Enter title: কালো সোনা খুজতে সাদা পাহাড়ের নিচে খনন
Enter content: একদল অসাধু কালো সোনা চোর তাদের সর্বশেষ অভিযানে সাদা পাহাড়ের নিচে গভীর রাতে সুরঙ্গ খুড়ে যাত্রা শুরু করলো অন্ধকারের পথে। হঠাত আধারের মাঝে জ্বলে উঠলো কালো সোনার গোলাপী আলো। তা দেখে সোনা চোরগুলো ঝাপিয়ে পড়ে কাপিয়ে দিলো। এই ছিলো সর্বশেষ সংবাদ!!!
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied third rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..


NB Prediction: Fake News 
LR Prediction: Fake News 
DT Prediction: Fake News 
KN Prediction: Fake News


In [ ]:
inp_domain = str(input("Enter domain: "))
inp_category = str(input("Enter category: "))
inp_source = str(input("Enter source: "))
inp_relation = str(input("Enter relation: "))
inp_title = str(input("Enter title: "))
inp_content = str(input("Enter content: "))
inp_news = inp_domain + ' ' + inp_category + ' ' + inp_source + ' ' + inp_relation + ' ' + inp_title + ' ' + inp_content
manual_testing(inp_news)

Enter domain: prothomalo.com
Enter category: International
Enter source: প্রথম আলো ডেস্ক
Enter relation: Related
Enter title: বাংলাদেশের স্বাধীনতার পক্ষে মোদির সত্যাগ্রহ ও কারাবরণের তথ্য নেই
Enter content: বাংলাদেশের স্বাধীনতা আন্দোলনের সমর্থনে নিজের সত্যাগ্রহ পালন ও কারাগারে যাওয়া বিষয়ে ভারতের প্রধানমন্ত্রী নরেন্দ্র মোদি যে বক্তব্য দিয়েছিলেন, তার কোনো তথ্য দিতে পারেনি দেশটির প্রধানমন্ত্রীর কার্যালয়।  ভারতের গণমাধ্যম দ্য অয়্যারের খবরে বলা হয়, দেশটির তথ্য অধিকার (আরটিআই) আইনে এ বিষয়ে জানতে চাওয়া হলে প্রধানমন্ত্রীর কার্যালয় মোদির গ্রেপ্তার ও মুক্তির কোনো নথি দিতে পারেনি।  বাংলাদেশের স্বাধীনতার ৫০ বছর পূর্তি ও মুজিব বর্ষের অনুষ্ঠানে যোগ দিতে গত বছরের ২৬ মার্চ বাংলাদেশে এসেছিলেন প্রধানমন্ত্রী নরেন্দ্র মোদি। সে সময় মোদি বলেন, (বাংলাদেশের) স্বাধীনতা আন্দোলনের সময় এ দেশের সমর্থনে সত্যাগ্রহ করেছিলেন তিনি। আর এ সত্যাগ্রহ করে জেলে যেতে হয়েছিল তাঁকে।  মোদির এ বক্তব্যের কড়া সমালোচনা করেছিলেন ভারতের বিরোধী দলের নেতারা। তিনি যে সত্যাগ্রহ করেছিলেন, সে–সংক্রান্ত নথিও তাঁর কাছে চেয়েছিলেন বিরোধীরা।  বাং